In [ ]:
import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
# import pandas as pd

## Multiclass classifier decision tree using ID3 algorithm

In [ ]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
#     transfer the data metrix to np array in float type.
    a = np.asarray(matrix)
    #print(a[:,-1])
    a = a.astype(float)
    #print("Before normalizing")
    #print(a)
#     apply the normalization along the 0 axis of a using the formula: (x - x_min)/(x_max - x_min)
    a = (a - a.min(axis=0))/(a.max(axis=0) - a.min(axis=0))
    #print("After normalizing")
    #print(a)
    return a
#matrix = load_csv('iris.csv')
#normalize(matrix[:,:4])
#normalize(matrix)

In [ ]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
    Z = np.genfromtxt(file, delimiter=',', dtype = str)
    return (Z)

# Z = load_csv("iris.csv")
# Z[0:10]

In [ ]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv2(file):
    Z = np.genfromtxt(file, delimiter=',', dtype = str, max_rows = 400, skip_header = 1600)
    return (Z)

# Z = load_csv("spambase.csv")
# Z[0:10]

In [ ]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
    np.random.shuffle(ar)
    return ar
#print(random_numpy_array(a))

In [ ]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
    #print(X.shape[0])
#     store the label X[:,-1] to Y
    Y = X[:,-1]
    #print(Y)
#     reshape y to (Y's length, 1)
    Y = Y.reshape(len(Y),1)
#     store it to j
    j = Y

    # print("J is",j)
#     create the new_X which exclude the label X[:,:-1]
    new_X = X[:,:-1]

#     normalize the data step
#     using our implemented function normalize()
    norm_X = normalize(new_X)

#     add the label back to the normiazlied X
#     using np.concatenate along axis=1
    X = np.concatenate((norm_X,j),axis=1)

#     store the size of rows of the normalized X with labels
    rows_X = X.shape[0]

#     use the 10% of the data to be the test set.
#     store the number of testing data
    num_test = math.floor(rows_X * 0.1)

#     set the starting idex to be 0
    start_idx = 0

#     set the ending index to be the number of testing data
    end_idx = num_test
#     create a list that store all features of the testing data
    test_features = []
#     create a list that store all labels of the testing data
    test_labels = []
#     create a list that store all features of the training data
    train_features = []
#     create a list that store all labels of the training data
    train_labels = []
#     10-fold cross-validation:
    for i in range(10):
#         store the test set for corss-validation using X[start:end,:]
      test = X[start_idx:end_idx,:]
#         get training data before the testing data X[:start, :]
      train_before_test = X[:start_idx, :]
#         get training data after the testing data X[:start, :]
      train_after_test = X[end_idx:, :]
#         form the new training set using np.concatenate
      train = np.concatenate((train_before_test, train_after_test))
#         get the testing set labels
      test_set_labels = test[:,-1]
#         flattent the labels
      test_set_labels.flatten()
#         get the training set labels
      train_set_labels = train[:,-1]
#         flattent the labels
      train_set_labels.flatten()
#         create the test set exclude the labels
      test = test[:,:-1]
      test = test.astype(np.float)
#         same for the training set
      train = train[:,:-1]
      train = train.astype(np.float)

#         append test data of this fold to the list
      test_features.append(test)
#         append test lables of this fold to the list
      test_labels.append(test_set_labels)
#         do the same for the training set
      train_features.append(train)
      train_labels.append(train_set_labels)

#         update the index pointer
      start_idx = end_idx
      end_idx += num_test

#     return the fold list that contain data and label for both training and testing set.
    return train_features, train_labels, test_features, test_labels

# matrix = load_csv('iris.csv')
# generate_set(matrix)

In [ ]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
#     init a dict for attributes
    dic = {} 
#     init feature list.
    feature = []
#     for each feature in the dataset
    #print(X)
    for i in range(X.shape[1]-1):
#         store the featur index
        feature_idx = i
#     find all the value correspond to this feature
        val = X[:,i]
        #print(val)
#     init an attribute list
        feature_attribute = []
#     init the counter to 0
        count = 0
#         for each value in the "all the value correspond to this feature"
        for j in val:
#             init a empty list that store the attribute value
            val_attribute = []
#             append the this value to the list
            val_attribute.append(j)
#             append the label of this value to the list
            val_attribute.append(y[count])
#             append this list to the attribute list.
            feature_attribute.append(val_attribute)
#             increase the counter
            count += 1
#         add this attribute list to the dict according to the feature index
        dic[feature_idx] = feature_attribute
#         append the feature indx to the feature list.
        feature.append(feature_idx)
#     return the dict and feature list.
    # print("dict: ", dic)
    # print("dict len", len(dic), len(dic[0]))
    return dic, feature

# X = load_csv("iris.csv")
# y = X[:,-1]
# build_dict_of_attributes_with_class_values(X,y)

In [ ]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
#     init a class frequence dict
    class_freq = {}
#     init the attribute entropy to 0
    attribute_entropy = 0
#     for each label in y:
    for i in y:
#         this is label is already in the dict, we increase its feq
        if i in class_freq:
            class_freq[i] += 1

#          else, we set the freq to 1
        else:
            class_freq[i] = 1

#     calculate the cumulate entropy using the formula.
    for i in class_freq:
        attribute_entropy += -(math.log(class_freq[i]/sum(class_freq[i] for i in class_freq)) * (class_freq[i]/sum(class_freq[i] for i in class_freq)))
    return attribute_entropy

#entropy(Z[:,-1])

In [ ]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,theta and leaf.
    def __init__(self, val, lchild, rchild,theta,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = theta
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self):
        return self.leaf

#     method to return threshold value
    def ret_thetha(self):
        return self.theta
    
#     method return root value
    def ret_root_value(self):
        return self.root_value 
    
#     method return left tree
    def ret_llist(self):
        return self.root_left

#     method return right tree
    def ret_rlist(self):
        return self.root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [ ]:

#Decision tree object
class DecisionTree(object):
#     init a variable called fea_list
    fea_list = []
#     init the Dtree by setting the root node to None
    def __init__(self):
        self.root_node = None

    #method to return the major class value using Counter() and .most_common()
    def cal_major_class_values(self,class_values):
        major_class_values = Counter(class_values).most_common(1)[0][0]
        return major_class_values

    #method to calculate best threshold value for each feature
    def cal_best_theta_value(self,ke,attri_list):
#         init a list for data
        # print(attri_list)
        data = []
#         init a list for class labes
        class_labels = []
#         for each attribute in the attri_list=
        for i in attri_list:
#             append the data
            #print(i[0])
            data.append(i[0])
#             append the feature value.
            class_labels.append(i[1])
#         calculate the entropy of those feaure values
        # data = [x.astype(np.float) for x in data]
        entropy_feature_values = entropy(class_labels)
#         init the max info gain = 0
        max_info_gain = 0
#         init theta=0
        theta = 0
#         init a list that store the best index on the left
        best_idx_left = []
#         init a list that store the best index on the right
        best_idx_right = []
#         init a list that store class labels after split
        class_labels_split = []
#         sort the data
        data.sort()
#         for each index of data:

        now = 0
        after = now + 1
        for i in range(len(data)-1):
#             calculate the current theta using data[i]+data[i+1])/ 2
        # print("data[i]: ", data[i])
        # print("type - data[i]: ", type(data[i]))
        # print("type - data[i+1]: ", type(data[i+1]))
        # print("data[i] + data[i+1]: ", data[i]+data[i+1])
            if data[now] != data[after]:
                current_theta = (float(data[now]) + float(data[after])) / 2.0
                now = after
                after += 1
            else:
                after += 1
                continue

  #             init a list that store index that less than theta
            idx_less_than_theta = []
  #             init a list that store value that less than theta
            val_less_than_theta = []
  #             init a list that store index that greater than theta
            idx_greater_than_theta = []
  #             init a list that store value that less than theta
            val_greater_than_theta = []
  #             init the counter to 0
            count = 0
  #             for each index and value in attri_list
            for c,j in enumerate(attri_list):
  #                 if value less or equal than the current theta:
                if (float(j[0]) <= float(current_theta)):
  #                     update the "less" list of index and value
                    idx_less_than_theta.append(c)
                    val_less_than_theta.append(j[1])
  #                 else:
                else:
  #                     update the "greater" list of index and value
                    idx_greater_than_theta.append(c)
                    val_greater_than_theta.append(j[1])

  #             calculate the entropy of the "less" list
            less_entropy = entropy(val_less_than_theta)
  #             calculate the entropy of the "greater" list
            greater_entropy = entropy(val_greater_than_theta)
  #             calculate the info gain using the formular.
  #             Entropy(Dataset) – (Count(Group1) / Count(Dataset) * Entropy(Group1) + Count(Group2) / Count(Dataset) * Entropy(Group2))
            info_gain = entropy_feature_values - (((len(idx_less_than_theta)/len(class_labels))*less_entropy)+((len(idx_greater_than_theta)/len(class_labels))*greater_entropy))
            
  #             if current info gain > max info gan
            if info_gain > max_info_gain:
  #                 update the info gain, 
                max_info_gain = info_gain
                theta = current_theta
  #                     the theta, the best index list of right, 
                best_idx_left = idx_less_than_theta
                best_idx_right = idx_greater_than_theta
  #                         the best index list of left and class_labels_list_after_split
                class_labels_split = val_less_than_theta + val_greater_than_theta

#         return the max info gain, theata,the best left list,the best right list and class label after split
        return max_info_gain, theta, best_idx_left, best_idx_right, class_labels_split

    #method to select the best feature out of all the features.
    def best_feature(self,dict_rep):
#         set key value to none
        key_value = None
#         set best info gain to -1
        best_info_gain = -1
#         set best theta to 0
        best_theta = 0
#         set best left list to empty
        best_left_list = []
#         set best right list to empty
        best_right_list = []
#         set best class labels after split to empty
        best_class_labels = []
#         init a result list
        result = []
#         for each key in dict_rep:
        for ke in dict_rep.keys():
#             using cal_best_theta_value() and store all returned values
            max_info_gain = self.cal_best_theta_value(ke,dict_rep[ke])[0]
            theta = self.cal_best_theta_value(ke,dict_rep[ke])[1]
            idx_left = self.cal_best_theta_value(ke,dict_rep[ke])[2]
            idx_right = self.cal_best_theta_value(ke,dict_rep[ke])[3]
            class_labels_list_after_split = self.cal_best_theta_value(ke,dict_rep[ke])[4] 
#             if info gian is greater than best info gain:
            if max_info_gain > best_info_gain:
#                 update info gain, theth, key value,
                best_info_gain = max_info_gain
                best_theta = theta
                key_value = ke
  
#                     left list, right list, class labels after split
                best_left_list = idx_left
                best_right_list = idx_right
                best_class_labels = class_labels_list_after_split

#         append the key value to the retrun list
        result.append(key_value)
        # result.append(best_info_gain)

#         append the theta value to the retrun list
        result.append(best_theta)

#         append the left list to the retrun list
        result.append(best_left_list)

#         append the right list to the retrun list
        result.append(best_right_list)

#         append the class labels to the retrun list
        result.append(best_class_labels)

#         return the list.
        #print(result)
        return  result

    def get_remainder_dict(self,dict_of_everything,index_split):
        global fea_list
#         init a split dict
        split_dict = {}
#         for each key "ke" in dict_of_everything:
        for ke in dict_of_everything.keys():
#             init a value list
            value_list = []
#             init a modified list
            modified = []
#             get the corresponding values of the key"ke" 
            values_of_ke = dict_of_everything[ke]
#             for each value and its corresponding index of the key"ke" 
            for idx in range(len(values_of_ke)):
#                 if index is not in the index_split:
                if idx not in index_split:
#                     append it to the modified list and value list
                    modified.append(values_of_ke[idx])
                    value_list.append(values_of_ke[idx][1])
#             add this modified list to the dict
            split_dict[ke] = modified
#         return the splited dict and val list
        return split_dict, value_list

    #method to create decision tree
    def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
        global fea_list
        #if all the class labels are same, then we are set
        if len(set(class_val)) == 1:
            #print("Leaf node for set class is",class_val[0],len(class_val))
            return Node(class_val[0], None, None, None, True)
        #if the no class vales are less than threshold, we assign the class with max values as the class label    
        elif len(class_val) < eta_min_val:
            return Node(self.cal_major_class_values(class_val), None, None, None, True)
#         else:
        else:
#             using the best_feature to get best feature list
            best_feature_list = self.best_feature(dict_of_everything)
#             store the node name, theta, left split, right split and class labes
            node_name = best_feature_list[0]
            theta = best_feature_list[1]
            left_split = best_feature_list[2]

            right_split = best_feature_list[3]
            
            class_label = best_feature_list[4]

#             call get_remainder_dict to get left tree data
            # left_dict = self.get_remainder_dict(dict_of_everything, left_split)[0]
            # left_class = self.get_remainder_dict(dict_of_everything, left_split)[1]
            left_dict, left_class = self.get_remainder_dict(dict_of_everything, left_split)

#             call get_remainder_dict to get right tree data
            # right_dict = self.get_remainder_dict(dict_of_everything, right_split)[0]
            # right_class = self.get_remainder_dict(dict_of_everything, right_split)[1]
            right_dict, right_class = self.get_remainder_dict(dict_of_everything, right_split)

#             call create_decision_tree to get left tree based on the left tree data
            left_tree = self.create_decision_tree(left_dict, left_class, eta_min_val)

#             call create_decision_tree to get right tree based on therightleft tree data
            right_tree = self.create_decision_tree(right_dict, right_class, eta_min_val)

        
#             set the root node
            root_node =  Node(node_name, right_tree, left_tree, theta, False)
            
#             return root node
            return root_node
            
    #fit the decisin tree
    def fit(self, dict_of_everything,cl_val,features,eta_min_val):
#         set the fea_list the value of features
        global fea_list
        fea_list = features
#         set the root node using the function create_decision_tree()
        self.root_node = self.create_decision_tree(dict_of_everything, cl_val, eta_min_val)
        return self.root_node

    def classify(self,row,root):
#         init the test dict
        dic = {}
#         add row to the dict
        for k,j in enumerate(row):
            dic[k] = j
#         set the current node to root
        current_node = root
#         while the current node is not leaf:
        while not current_node.leaf:
#             implement the case whether the current shoud go to the left
            if float(dic[current_node.root_value]) <= float(current_node.theta):
                current_node  = current_node.root_left
#             implement the case whether the current shoud go to the right
            else:
              current_node = current_node.root_right
#         return the calss of the current node
        return current_node.root_value
        
    # method to the labels for the test data
    def predict(self, X, root):
#         predict using the classify()
        l = [self.classify(row,root) for row in X]
        return l

# D1 = DecisionTree()
# dic_rep, feature_list = build_dict_of_attributes_with_class_values(X,y)
# class_values = y
# eta_min_val = 10
# D1.create_decision_tree(dic_rep, class_values, eta_min_val)


In [ ]:
def accuracy_for_predicted_values(test_class_names1,l):
#     init true and false count to 0
    true, false = 0, 0
#     for each prediction,if predict is correct then, true++ else, false++
    for i in range(len(test_class_names1)):
        if test_class_names1[i] == l[i]:
            true += 1
        else:
            false += 1
    acc = true / (true + false)
#     return the acc
    return acc

In [ ]:
def main(num_arr, eta_min):
    eta_min_val = round(eta_min*num_arr.shape[0])

    #randomly shuffle the array so that we can divide the data into test/training
    num_arr = random_numpy_array(num_arr)
    
    #divide data into ztest labels,test features,training labels, training features
    train_features, train_labels, test_features, test_labels = generate_set(num_arr)

    #init cumulate acc to 0
    cumulate_acc = 0

    # ten fold iteration 
    # print("train_features: ",train_features)
    # print("train_features[0]",train_features[0])

    for i in range(10):
        #build a dictionary with class labels and respective features values belonging to that class
        attri_dict, feature_list = build_dict_of_attributes_with_class_values(train_features[i], train_labels[i])
        
        #instantiate decision tree instance
        dTree = DecisionTree()
        
        # build the decision tree model.
        dTree.fit(attri_dict, train_labels[i], feature_list, eta_min_val)
        
        #predict the class labels for test features
        #print("root_node: ", dTree.root_node)
        predict_labels = dTree.predict(test_features[i], dTree.root_node)
        #print("predict_labels: ", predict_labels)
        
        #calculate the accuracy for the predicted values
        acc = accuracy_for_predicted_values(predict_labels, test_labels[i])
        
        #add acc to cumulate acc
        cumulate_acc += acc

        print("Accuracy: ", accuracy_for_predicted_values(predict_labels, test_labels[i]))
    print("Accuracy across 10-cross validation for", eta_min, "is", float(cumulate_acc)/10, "\n")

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
  main(num_arr,i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


Accuracy:  1.0
Accuracy:  0.8666666666666667
Accuracy:  0.8666666666666667
Accuracy:  1.0
Accuracy:  0.9333333333333333
Accuracy:  0.9333333333333333
Accuracy:  0.9333333333333333
Accuracy:  0.8666666666666667
Accuracy:  1.0
Accuracy:  1.0
Accuracy across 10-cross validation for 0.05 is 0.9400000000000001 

Accuracy:  0.9333333333333333
Accuracy:  1.0
Accuracy:  0.8666666666666667
Accuracy:  1.0
Accuracy:  0.8
Accuracy:  1.0
Accuracy:  0.8666666666666667
Accuracy:  0.8666666666666667
Accuracy:  0.9333333333333333
Accuracy:  1.0
Accuracy across 10-cross validation for 0.1 is 0.9266666666666667 

Accuracy:  0.8666666666666667
Accuracy:  0.8666666666666667
Accuracy:  0.8666666666666667
Accuracy:  0.9333333333333333
Accuracy:  0.8
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  0.9333333333333333
Accuracy:  1.0
Accuracy across 10-cross validation for 0.15 is 0.9266666666666665 

Accuracy:  1.0
Accuracy:  0.8666666666666667
Accuracy:  0.9333333333333333
Accuracy:  0.9333333333333333

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "spambase.csv"
num_arr = load_csv2(newfile)
for i in eta_min_list:
  main(num_arr,i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


Accuracy:  0.975
Accuracy:  0.875
Accuracy:  0.975
Accuracy:  0.95
Accuracy:  0.9
Accuracy:  0.95
Accuracy:  0.925
Accuracy:  0.875
Accuracy:  0.95
Accuracy:  0.875
Accuracy across 10-cross validation for 0.05 is 0.925 

Accuracy:  0.95
Accuracy:  0.875
Accuracy:  0.9
Accuracy:  0.925
Accuracy:  0.875
Accuracy:  0.95
Accuracy:  0.925
Accuracy:  0.825
Accuracy:  0.975
Accuracy:  0.9
Accuracy across 10-cross validation for 0.1 is 0.9100000000000001 

Accuracy:  0.9
Accuracy:  0.925
Accuracy:  0.925
Accuracy:  0.875
Accuracy:  0.925
Accuracy:  0.925
Accuracy:  0.95
Accuracy:  0.925
Accuracy:  0.975
Accuracy:  0.9
Accuracy across 10-cross validation for 0.15 is 0.9225 

Accuracy:  0.95
Accuracy:  0.95
Accuracy:  0.95
Accuracy:  0.95
Accuracy:  0.925
Accuracy:  0.9
Accuracy:  0.8
Accuracy:  0.875
Accuracy:  0.975
Accuracy:  0.95
Accuracy across 10-cross validation for 0.2 is 0.9225 

Accuracy:  0.975
Accuracy:  0.875
Accuracy:  0.925
Accuracy:  0.975
Accuracy:  0.9
Accuracy:  0.975
Accuracy